# Turing Machines

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
import sys

# -- Detect if in Own Install or in Colab
try:
    import google.colab
    OWN_INSTALL = False
except:
    OWN_INSTALL = True
    
if OWN_INSTALL:
    
  #---- Leave these definitions ON if running on laptop
  #---- Else turn OFF by putting them between ''' ... '''

  sys.path[0:0] = ['../../../../..',  '../../../../../3rdparty',  
                   '../../../..',  '../../../../3rdparty',  
                   '../../..',     '../../../3rdparty', 
                   '../..',        '../../3rdparty',
                   '..',           '../3rdparty' ]

else: # In colab
  ! if [ ! -d Jove ]; then git clone https://github.com/ganeshutah/Jove Jove; fi
  sys.path.append('./Jove')
  sys.path.append('./Jove/jove')

# -- common imports --
from jove.Def_TM     import *
from jove.Def_md2mc  import *
from jove.DotBashers import *
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

In [ ]:
 # DTM to locate "101"

dtm101 = md2mc('''TM
I : 0; A, R -> I
I : 1; B, R -> Got1Sk0
I : .; ., R -> StuckNo1beg

Got1Sk0 : 1; B, R -> Got1Sk0
Got1Sk0 : .; ., R -> StuckNo0Aft1
Got1Sk0 : 0; A, R -> Got10Sk1

Got10Sk1 : 0; A, R -> I
Got10Sk1 : 1; B, R -> Found101
Got10Sk1 : .; ., R -> StuckNo1end

''')
DO_dtm101 = dotObj_tm(dtm101, FuseEdges=True)
DO_dtm101.render('dtm101')
DO_dtm101

In [ ]:
explore_tm(dtm101, "00101", 10)

In [ ]:
# NDTM to locate "101"

ndtm101 = md2mc('''TM

I : 0; A, R | 1; B, R -> I
I : 0; 0, S | 1; 1, S -> TryMyLuck

TryMyLuck : 1; Q, R -> Got1Sk0
Got1Sk0   : 0; P, R -> Got10Sk1
Got10Sk1  : 1; Q, R -> Found101

I : .; ., R -> Reject
 
''')
DO_ndtm101 = dotObj_tm(ndtm101, FuseEdges=True)
DO_ndtm101.render('ndtm101')
DO_ndtm101

In [ ]:
explore_tm(ndtm101, "00101", 10)

## This TM (written by Ian Briggs) doubles a number given in decimal

In [ ]:
dec_doub = md2mc(
"""
TM
i_start : 0;.,R -> erase_leading_0
i_start : 1;1,R | 2;2,R | 3;3,R | 4;4,R | 5;5,R | 6;6,R | 7;7,R | 8;8,R | 9;9,R -> goto_lsd
i_start : .;.,S -> error

erase_leading_0 : 0;.,R -> erase_leading_0
erase_leading_0 : 1;1,R | 2;2,R | 3;3,R | 4;4,R | 5;5,R | 6;6,R | 7;7,R | 8;8,R | 9;9,R -> goto_lsd
erase_leading_0 : .;0,S -> f_done

goto_lsd : 0;0,R | 1;1,R | 2;2,R | 3;3,R | 4;4,R | 5;5,R | 6;6,R | 7;7,R | 8;8,R | 9;9,R -> goto_lsd
goto_lsd : .;.,L -> 0_carry

0_carry : 0;0,L | 1;2,L | 2;4,L | 3;6,L | 4;8,L -> 0_carry
0_carry : 5;0,L | 6;2,L | 7;4,L | 8;6,L | 9;8,L -> 1_carry
0_carry : .;.,S -> f_done

1_carry : 0;1,L | 1;3,L | 2;5,L | 3;7,L | 4;9,L -> 0_carry
1_carry : 5;1,L | 6;3,L | 7;5,L | 8;7,L | 9;9,L -> 1_carry
1_carry : .;1,S -> f_done
!!"""

)
    
# -- src='File', fname='../../machines/tmfiles/decimal_double_tm.tm'


In [ ]:
dotObj_tm(dec_doub, FuseEdges = True)

In [ ]:
# Doubles the number given on the tape in decimal!
explore_tm(dec_doub, "231", 100)

### This is also known as the Collatz Problem which is to check whether a function of the following form
### terminates on all inputs. We give the Collatz problem in Python below.

In [ ]:
def collatz(x):
    if (x%2 == 0): 
        print(x, ", ",end="")
        return collatz(x//2)
    elif (x==1):
        print(x, ", ",end="")
        return 1
    else:
        print(x, ", ",end="")
        return collatz(3*x + 1)

In [ ]:
collatz(23)

In [ ]:
collatz(73)

## This TM (written by Ian Briggs) runs the code associated with the "3x+1" problem 

In [ ]:
'''
(src='File', fname='../../machines/tmfiles/collatz_tm.tm')
'''

collatz_tm = md2mc(
"""
TM

i_start      : 0; ., R -> i_start             !! erase this zero and try to find more
i_start      : 1; 1, R -> goto_lsb            !! we have a proper number, go to the lsb
i_start      : .; ., S -> error               !! error on no input or input == 0


goto_lsb     : 0; 0,R | 1; 1,R -> goto_lsb    !! scan off the right edge of the number
goto_lsb     : .; .,L -> branch               !! take a step back to be on the lsb and start branch


branch       : 0; .,L -> branch               !! number is even, divide by two and re-branch
branch       : 1; 1,L -> check_n_eq_1         !! number is odd, check if it is 1


check_n_eq_1 : 0; 0,R | 1; 1,R -> 01_fma      !! number wasn't 1, goto 3n+1
check_n_eq_1 : .; .,R -> f_halt               !! number was 1, halt


!! carrying 0 we see a 0 so write 0 and carry 0 forward
00_fma       : 0; 0,L -> 00_fma

!! carrying 0 we see a 1 (times 3 is 11) so write 1 and carry 1 forward
00_fma       : 1; 1,L -> 01_fma

!! reached the end of the number, go back to the start
00_fma       : .; .,R -> goto_lsb             


!! carrying 1 we see a 0 so write 1 and carry 0 forward
01_fma       : 0; 1,L -> 00_fma  

!! carrying 1 we see a 1 (times 3 is 11, plus our carry is 100) so write 0 and carry 10 forward
01_fma       : 1; 0,L -> 10_fma  

!! reached the end of the number, write our 1 and go back to the start
01_fma       : .; 1,R -> goto_lsb             


!! carrying 10 we see a 0, so write 0 and carry 1 forward
10_fma       : 0; 0,L -> 01_fma

!! carrying 10 we see a 1 (times 3 is 11, plus our carry is 101), so write 1 and carry 10 forward
10_fma       : 1; 1,L -> 10_fma

!! reached the end of the number, write a 0 from our 10 and carry 1
10_fma       : .; 0,L -> 01_fma

"""
)

In [ ]:
DOcollatz = dotObj_tm(collatz_tm, FuseEdges=True)
DOcollatz

In [ ]:
# Will loop if the Collatz ("3x+1") program will ever loop!
explore_tm(collatz_tm, "0110", 100)

### This TM written by Ian Briggs shifts the given string of a's and b's
### puts a gap before the last character and shifts the other characters
### one place left. E.g., ababba -> ababb.a

In [ ]:
'''(src='File', fname='../../machines/tmfiles/shift_left_tm.tm')
'''

shiftl_tm = md2mc(
"""
!! Input: a string matching the regex "[ab]*"
!!
!! Output: the same input string with a gap before the last character
!!
!! Errors: None

TM
i_start : a;a,R | b;b,R -> scan_to_end
i_start : .;.,S -> f_done

scan_to_end : a;a,R | b;b,R -> scan_to_end
scan_to_end : .;.,L -> move_left

move_left : a;a,L | b;b,L -> read

read : a;.,L -> a_next
read : b;.,L -> b_next

a_next : a;a,L -> a_next
a_next : b;a,L -> b_next
a_next : .;a,L -> f_done

b_next : a;b,L -> a_next
b_next : b;b,L -> b_next
b_next : .;b,L -> f_done

"""

)

In [ ]:
dotObj_tm(shiftl_tm, FuseEdges=True)

In [ ]:
explore_tm(shiftl_tm, "abaaba", 100)

### This TM written by Ian Briggs shifts the given string of a's and b's
### puts a gap before the last character and shifts the other characters
### one place left. E.g., ababba -> a.babba

In [ ]:
'''
(src='File', fname='../../machines/tmfiles/shift_right_tm.tm')
'''

shiftr_tm = md2mc(
"""
!! Input: a string matching the regex "[ab]*"
!!
!! Output: the same input string with a gap after the first character
!!
!! Errors: None
TM
i_move_right : a;a,R | b;b,R -> read
i_move_right : .;.,S -> f_done

read : a;.,R -> a_next
read : b;.,R -> b_next
read : .;.,S -> f_done

a_next : a;a,R -> a_next
a_next : b;a,R -> b_next
a_next : .;a,S -> f_done

b_next : a;b,R -> a_next
b_next : b;b,R -> b_next
b_next : .;b,S -> f_done
!!"""

)

In [ ]:
dotObj_tm(shiftr_tm, True)

In [ ]:
explore_tm(shiftr_tm, "ababba", 100)

# TM that performs bit-serial addition is below

## Written by Ian Briggs

In [ ]:
# The TM with input such as 

# 0101+0110= where 0101 and 0110 are presented little-endian


# Assume that the inputs will be padded to be the same length
#
# The example addition being performed is (written in the big-endian manner) is:
#
# 0110
# 1010
# ----
# 10000
#


# Algorithm : State design is of course key!
#
# We are also in various states. Here is how we name the states:
#
# B means before
# B+ means we are before the + 
# B= means we are before the =

# B+p means we are BEFORE the + 
#   as well as PRIOR to reading a 0 and changing it to an X or reading a 1 and changing it to a Y

# B+q means we are AFTER the + and after changing 0->X or 1->Y

# B=p and B=q have the same significance

# Finally the _00 or _01 or _10 or _11 means
# _<carry><sum>
# i.e. the present carry and the present sum are recorded after the underscore

addtm = md2mc('''TM 
 
I_00   : 0;0,S | 1;1,S  -> B+p_00 !! From init state, go to Before +, prior 

B+p_00 : 0;X,R          -> B+q_00  !! Change to Before +, after
B+p_00 : 1;Y,R          -> B+q_01

B+p_00 : X;X,R | Y;Y,R  -> B+p_00  !! Skip over X,Y
B+p_00 : +;+,R          -> EnsureSndDone0   !! Carry=0; Maybe addition is over; check second piece also "done"
 
B+p_10 : 0;X,R          -> B+q_01
B+p_10 : 1;Y,R          -> B+q_10

B+p_10 : X;X,R | Y;Y,R  -> B+p_10
B+p_10 : +;+,R          -> EnsureSndDone1   !! Carry=1; similar to EnsureSndDone0 otherwise

B+q_00 : 0;0,R  | 1;1,R -> B+q_00
B+q_00 : +;+,R          -> B=p_00  !! Move onto the "Before =" part

B+q_01 : +;+,R          -> B=p_01
B+q_01 : 0;0,R  | 1;1,R -> B+q_01

B+q_10 : 0;0,R  | 1;1,R -> B+q_10
B+q_10 : +;+,R          -> B=p_10

 
!!---

B=p_00 : X;X,R | Y;Y,R -> B=p_00  !! skip over the initial X and Y
B=p_00 : 0;X,R         -> B=q_00  !! We have reached the "after" part
B=p_00 : 1;Y,R         -> B=q_01
 
B=p_10 : X;X,R | Y;Y,R -> B=p_10
B=p_10 : 0;X,R         -> B=q_10
B=p_10 : 1;Y,R         -> B=q_11

B=q_11 : 0;0,R | 1;1,R -> B=q_11
B=q_11 : =;=,R         -> SkEnd_11

B=p_01 : X;X,R | Y;Y,R -> B=p_01
B=q_01 : =;=,R         -> SkEnd_01        !! Seeking end

B=q_00 : 0;0,R | 1;1,R -> B=q_00
B=q_00 : =;=,R         -> SkEnd_00

B=q_01 : 0;0,R | 1;1,R -> B=q_01
B=q_10 : 0;0,R | 1;1,R -> B=q_10

B=q_10 : =;=,R         -> SkEnd_10
B=p_01 : 0;X,R         -> B=q_01
B=p_01 : 1;Y,R         -> B=q_10

 
!!---

SkEnd_00 : 0;0,R | 1;1,R -> SkEnd_00
SkEnd_00 : .;0,L         -> RstHdL0   !! Reset head to the left; carry=0

SkEnd_01 : 0;0,R | 1;1,R -> SkEnd_01
SkEnd_01 : .;1,L         -> RstHdL0

SkEnd_10 : 0;0,R | 1;1,R -> SkEnd_10
SkEnd_10 : .;0,L         -> RstHdL1   !! Reset head to the left; carry=1

SkEnd_11 : 0;0,R | 1;1,R -> SkEnd_11
SkEnd_11 : .;1,L         -> RstHdL1

 
!!---

EnsureSndDone0 : X;X,R | Y;Y,R -> EnsureSndDone0 !! Ensure done, carry=0
EnsureSndDone0 : =;=,R         -> F       !! Yes, second piece also properly done

EnsureSndDone1 : X;X,R | Y;Y,R -> EnsureSndDone1 !! Ensure done, carry=1
EnsureSndDone1 : =;=,R         -> Last1          !! Must stick in last 1 in MSB of result

 
!!---
 
Last1 : 0;0,R | 1;1,R  -> Last1           !! Stick in 1 in MSB of result
Last1 : .;1,R          -> F               !! drain pending carry as there are no more bits to add


!!---

RstHdL0  : 0;0,L | 1;1,L | X;X,L | Y;Y,L | +;+,L | =;=,L -> RstHdL0
RstHdL1  : 0;0,L | 1;1,L | X;X,L | Y;Y,L | +;+,L | =;=,L -> RstHdL1

RstHdL0  : .;.,R -> B+p_00
RstHdL1  : .;.,R -> B+p_10

''')
DOaddtm = dotObj_tm(addtm, FuseEdges=True)
DOaddtm.render('addtm')
DOaddtm

In [ ]:
explore_tm(addtm,"11+11=",100)

In [ ]:
explore_tm(addtm,"0101+0110=",140)

In [ ]:
explore_tm(addtm,"10000+11111=",200)

# END